In [2]:
import pandas as pd
import json, os
from tqdm import tqdm

The following table represents URLs parsed from https://www.proteinatlas.org/download/proteinatlas.xml.gz, merged against https://www.proteinatlas.org/download/normal_tissue.tsv.zip and https://www.proteinatlas.org/download/rna_tissue_consensus.tsv.zip.

In [3]:
# df = pd.read_csv('./data/hpa_v21_kidney.csv',index_col=0)
df = pd.read_csv('./data/hpa_v21.csv.gz',index_col=0).query('Tissue=="testis"')
df

,Gene name,Gene,UniProt,Antibody,Tissue,Staining,Sex,Age,Patient,URL,Batch,Slide,Well,Reliability,nTPM
Image,,,,,,,,,,,,,,,
11813_A_4_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,26.0,2071,http://images.proteinatlas.org/4109/11813_A_4_...,4109,11813,A_4_6,Approved,75.5
11813_A_5_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,68.0,1880,http://images.proteinatlas.org/4109/11813_A_5_...,4109,11813,A_5_6,Approved,75.5
11813_A_6_6,TSPAN6,ENSG00000000003,O43657,HPA004109,testis,low,Male,46.0,2435,http://images.proteinatlas.org/4109/11813_A_6_...,4109,11813,A_6_6,Approved,75.5
132061_A_4_6,TNMD,ENSG00000000005,Q9H2S6,HPA034961,testis,medium,Male,13.0,2037,http://images.proteinatlas.org/34961/132061_A_...,34961,132061,A_4_6,NaN,0.1
132061_A_5_6,TNMD,ENSG00000000005,Q9H2S6,HPA034961,testis,medium,Male,25.0,3627,http://images.proteinatlas.org/34961/132061_A_...,34961,132061,A_5_6,NaN,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166208_A_5_6,AC013477.1,ENSG00000288637,NaN,HPA043499,testis,not detected,Male,65.0,2558,http://images.proteinatlas.org/43499/166208_A_...,43499,166208,A_5_6,NaN,0.0
166208_A_6_6,AC013477.1,ENSG00000288637,NaN,HPA043499,testis,not detected,Male,41.0,4485,http://images.proteinatlas.org/43499/166208_A_...,43499,166208,A_6_6,NaN,0.0
170537_A_4_6,AL353671.1,ENSG00000288684,NaN,HPA073217,testis,high,Male,29.0,4845,http://images.proteinatlas.org/73217/170537_A_...,73217,170537,A_4_6,NaN,0.1


Here we download each image, pad or crop to a multiple of 512x512, downscale to 512x512, and cast to 8-bit PNG. We also associate with each a JSON file containing its metadata from the above table.

In [ ]:
TISSUE = 'testis'
DOWNLOAD = False

for _, row in tqdm(df.reset_index().iterrows(),position=0):
    image = row['Image']
    batch = row['Batch']
    url = row['URL']
    path = f'./data/images/{TISSUE}/{batch}'
    !mkdir -p {path}
    if not DOWNLOAD:
        if not os.path.exists(f'{path}/{image}.png'): 
            continue
    else:
        !curl -so {path}/{image}.jpg "{url}"'
        !mogrify -extent 3072x3072 -gravity center -scale 512x512 -format png8 {path}/{image}.jpg
        !rm {path}/{image}.jpg
        !mv {path}/{image}.png8 {path}/{image}.png
    with open(f'{path}/{image}.json','w') as f:
        json.dump(row.to_dict(), f)

46153it [2:42:01,  5.08it/s]